In [ ]:
!pip install pyngrok flask-ngrok tensorflow pillow

ส่วนที่ 1 การนำเข้าข้อมูล

In [ ]:
from flask import Flask, request, render_template_string, make_response, redirect, url_for, send_from_directory, session
from google.colab import drive
import os
import pandas as pd
from PIL import Image
import numpy as np
import json
from pyngrok import ngrok
import tensorflow as tf
import shutil
import random
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout




drive.mount('/content/drive')



base_path = '/content/drive/MyDrive/Face_Recog_App'
user_img_dir = f'{base_path}/static/uploads/UserImages'
os.makedirs(user_img_dir, exist_ok=True)

csv_file = f'{user_img_dir}/users.csv'
if not os.path.exists(csv_file):
    df = pd.DataFrame(columns=['name', 'surname', 'phone', 'folder'])
    df.to_csv(csv_file, index=False)

Mounted at /content/drive


ส่วนที่ 2 โหลดภาพเป็น RGB

In [ ]:
# ฟังก์ชันโหลดข้อมูลภาพ (RGB 3 channels)
def load_data_rgb(data_path, img_size=(100,100)):
    images = []
    labels = []
    label_map = {}
    label_id = 0

    for folder in os.listdir(data_path):
        folder_path = os.path.join(data_path, folder)
        if not os.path.isdir(folder_path):
            continue
        if folder not in label_map:
            label_map[folder] = label_id
            label_id += 1
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path)  # อ่านสี (BGR)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # แปลงเป็น RGB
                img = cv2.resize(img, img_size)
                images.append(img)
                labels.append(label_map[folder])

    images = np.array(images, dtype=np.float32) / 255.0  # Normalize
    labels = np.array(labels)
    return images, labels, label_map

ส่วนที่ 3 หน้ารวม html


In [ ]:
app = Flask(__name__)

#หน้า register
REGISTER_HTML = """
<!DOCTYPE html>
<html lang="th">
<head>
  <meta charset="UTF-8">
  <title>ลงทะเบียนผู้ใช้</title>
  <link href="https://fonts.googleapis.com/css2?family=Prompt&display=swap" rel="stylesheet">
  <style>
    body {
      font-family: 'Prompt', sans-serif;
      background-color: #f0f4f8;
      color: #333;
      padding: 20px;
    }
    h2 {
      color: #2c3e50;
      text-align: center;
    }
    form {
      background: white;
      padding: 50px;
      border-radius: 12px;
      max-width: 400px;
      margin: auto;
      box-shadow: 0 4px 10px rgba(0,0,0,0.1);
    }
    small#angle {
  font-size: 1.5em;
  font-weight: bold;
}

    input, button {
      padding: 10px;
      margin-top: 10px;
      width: 100%;
      border-radius: 6px;
      border: 1px solid #ccc;
    }
    button {
      background-color: #3498db;
      color: white;
      cursor: pointer;
    }
    button:hover {
      background-color: #2980b9;
    }
    video {
      display: block;
      margin: 10px auto;
      border-radius: 12px;
    }
    small {
      display: block;
      text-align: center;
      margin-top: 10px;
    }
  </style>
</head>
<body>

<h2>ลงทะเบียนผู้ใช้</h2>
<form id="regForm">
  <label>ชื่อ: <input type="text" name="name" required></label><br>
  <label>นามสกุล: <input type="text" name="surname" required></label><br>
  <label>เบอร์โทร: <input type="text" name="phone" required></label><br><br>
  <button type="button" id="toggleBtn" onclick="toggleCamera()"> เปิดกล้อง</button><br><br>
  <video id="video" width="320" height="240" autoplay></video><br>
  <button type="button" onclick="capture()"> ถ่ายภาพ</button><br>
  <small id="angle">มุมที่ 1/15: มองตรง</small><br>
  <button type="submit"> ส่งข้อมูล</button>
  <a href="/">
    <button type="button"> ย้อนกลับไปหน้า Login</button>
  </a>
</div>
</form>

<script>
  const video = document.getElementById('video');
  const angleText = document.getElementById('angle');
  const toggleBtn = document.getElementById('toggleBtn');
  const angles = ["มองตรง", "หันซ้ายเล็กน้อย", "หันขวาเล็กน้อย", "เงยหน้า", "ก้มหน้า",
    "เอียงซ้าย", "เอียงขวา", "หันซ้าย", "หันขวา", "ยิ้ม", "หน้าตึง", "หลับตา",
    "เปิดตากว้าง", "ขยับคิ้ว", "มองขึ้น"];
  let images = [];
  let current = 0;
  let mediaStream = null;
  let isCameraOn = false;

  async function toggleCamera() {
    if (isCameraOn) {
      mediaStream.getTracks().forEach(track => track.stop());
      video.srcObject = null;
      video.style.display = "none";
      toggleBtn.textContent = " เปิดกล้อง";
      isCameraOn = false;
    } else {
      try {
        mediaStream = await navigator.mediaDevices.getUserMedia({ video: true });
        video.srcObject = mediaStream;
        video.style.display = "block";
        toggleBtn.textContent = " ปิดกล้อง";
        isCameraOn = true;
      } catch (err) {
        alert(" ไม่สามารถเปิดกล้องได้: " + err);
      }
    }
  }

  function capture() {
    if (!isCameraOn) {
      alert(" กรุณาเปิดกล้องก่อน");
      return;
    }
    if (current >= angles.length) return;
    const canvas = document.createElement('canvas');
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;
    canvas.getContext('2d').drawImage(video, 0, 0);
    canvas.toBlob(blob => {
      images.push(blob);
      current++;
      if (current < angles.length) {
        angleText.textContent = `มุมที่ ${current + 1}/15: ${angles[current]}`;
      } else {
        angleText.textContent = " ถ่ายครบแล้ว กด 'ส่งข้อมูล'";
      }
    }, 'image/jpeg');
  }

  document.getElementById('regForm').onsubmit = e => {
    e.preventDefault();
    if (images.length < angles.length) {
      alert("กรุณาถ่ายภาพครบ 15 มุมก่อนส่งข้อมูล");
      return;
    }
    const formData = new FormData(e.target);
    images.forEach((img, i) => formData.append('image_' + i, img, `img_${i + 1}.jpg`));
    fetch('/register', { method: 'POST', body: formData })
    .then(res => {
  if (res.redirected) {
    window.location.href = res.url;
  } else {
    return res.text().then(text => alert(text));
  }
})

      .catch(err => alert("เกิดข้อผิดพลาด: " + err));
  };
</script>


</body>
</html>
"""
#องค์ประกอบต่างๆของหน้า profile
PROFILE_HTML = """
<!DOCTYPE html>
<html lang="th">
<head>
  <meta charset="UTF-8">
  <title>โปรไฟล์ผู้ใช้</title>
  <style>
    body {
      margin: 0;
      padding: 0;
      background-color: #f4f6f9;
      font-family: 'Segoe UI', sans-serif;
      display: flex;
      justify-content: center;
      align-items: flex-start;
      min-height: 100vh;
    }

    .container {
      max-width: 800px;
      width: 100%;
      background-color: #fff;
      margin: 40px auto;
      padding: 32px;
      border-radius: 12px;
      box-shadow: 0 4px 12px rgba(0, 0, 0, 0.08);
    }

    h2, h3, h4 {
      color: #333;
      margin-bottom: 12px;
    }

    p {
      color: #555;
      line-height: 1.6;
      font-size: 16px;
    }

    b {
      color: #222;
    }

    .main-image {
      width: 100%;
      max-width: 400px;
      height: 400px;
      object-fit: cover;
      border-radius: 12px;
      margin: 20px auto;
      display: block;
      box-shadow: 0 0 8px rgba(0, 0, 0, 0.05);
    }

    .gallery {
      display: flex;
      flex-wrap: wrap;
      gap: 10px;
      margin-top: 16px;
    }

    .gallery img {
      width: 100px;
      height: 100px;
      object-fit: cover;
      border-radius: 8px;
      box-shadow: 0 1px 4px rgba(0, 0, 0, 0.1);
      transition: transform 0.2s ease;
    }

    .gallery img:hover {
      transform: scale(1.05);
    }

    a {
      display: inline-block;
      margin-top: 30px;
      text-decoration: none;
      color: #2c3e50;
      font-size: 14px;
    }

    a:hover {
      text-decoration: underline;
    }

  </style>
</head>
<body>
  <div class="container">
    <h2>หน้าโปรไฟล์ผู้ใช้</h2>
    <p><b>ชื่อ:</b> {{ user.name }}</p>
    <p><b>นามสกุล:</b> {{ user.surname }}</p>
    <p><b>เบอร์โทร:</b> {{ user.phone }}</p>

    <h3>รูปภาพของคุณ</h3>

    <h4> รูปสุดเซ่กซี่:</h4>
    <div>
      {% for image_name in images %}
        {% if image_name == 'img_15.jpg' %}
          <img src="/static/uploads/UserImages/{{ user.folder }}/{{ image_name }}" class="main-image">
        {% endif %}
      {% endfor %}
    </div>

    <h4> มุมอื่น ๆ:</h4>
    <div class="gallery">
      {% for image_name in images %}
        {% if image_name != 'img_15.jpg' %}
          <img src="/static/uploads/UserImages/{{ user.folder }}/{{ image_name }}">
        {% endif %}
      {% endfor %}
    </div>

<form method="POST" action="/delete_account"
      onsubmit="return confirm('คุณแน่ใจหรือไม่ว่าต้องการลบบัญชีนี้?');"
      style="margin-top: 30px; text-align: center;">
  <input type="hidden" name="phone" value="{{ user.phone }}">
  <button type="submit"
          style="padding: 10px 20px; border: none; background-color: #e74c3c; color: white; font-weight: bold; border-radius: 6px; font-size: 14px; cursor: pointer;">
     ลบบัญชีนี้
  </button>
</form>

    <a href="/">← กลับสู่หน้า Login

  </button>
</form></a>
  </div>
</body>
</html>
"""

#HTML เป็นตกเเต่งของหน้าหลัก หรือ หน้า home  องค์ประกอบของหน้า หลัก
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Face Login</title>
  <style>
    body {
      display: flex;
      justify-content: center;
      align-items: center;
      flex-direction: column;
      height: 100vh;
      margin: 0;
      font-family: 'Segoe UI', sans-serif;
      background-color: #f0f4f8;
    }

    h2 {
      margin-bottom: 20px;
      color: #333;
    }

    form, .container {
      background-color: #fff;
      padding: 24px 32px;
      border-radius: 12px;
      box-shadow: 0 4px 12px rgba(0,0,0,0.1);
      display: flex;
      flex-direction: column;
      align-items: center;
      gap: 14px;
    }

    input[type="text"] {
      padding: 10px;
      width: 100%;
      max-width: 300px;
      border-radius: 6px;
      border: 1px solid #ccc;
      font-size: 14px;
    }

    video {
      border-radius: 8px;
      border: 2px solid #ccc;
    }

    button {
      padding: 10px 20px;
      font-size: 14px;
      border-radius: 6px;
      border: none;
      cursor: pointer;
    }

    button:first-of-type {
      background-color: #2e89ff;
      color: white;
    }

    button:first-of-type:hover {
      background-color: #176fe0;
    }

    button:last-of-type {
      background-color: #e0e0e0;
      color: #333;
    }

    a {
      text-decoration: none;
      color: #c0392b;
      margin-top: 10px;
    }

    #result {
      margin-top: 10px;
      color: #555;
      font-size: 14px;
    }
  </style>
</head>
<body>
  <div class="container">
    <h2>Face Login</h2>
    <input type="text" id="phone" placeholder="เบอร์โทรศัพท์">
    <video id="video" width="320" height="240" autoplay></video>
    <button id="toggleButton" onclick="toggleCamera()"> เปิดกล้อง</button>
    <button onclick="capture()"> Capture & Login</button>
    <button onclick="location.href='/register'"> ลงทะเบียนผู้ใช้</button>
    <canvas id="canvas" style="display: none;"></canvas>
    <p id="result">{{ result }}</p>
  </div>

  <script>
  const video = document.getElementById('video');
  const canvas = document.getElementById('canvas');
  const toggleBtn = document.getElementById('toggleButton');
  let mediaStream = null;
  let isCameraOn = false;

  async function startCamera() {
    try {
      mediaStream = await navigator.mediaDevices.getUserMedia({ video: true });
      video.srcObject = mediaStream;
      isCameraOn = true;
      toggleBtn.textContent = " ปิดกล้อง";
    } catch (err) {
      alert(" ไม่สามารถเปิดกล้องได้: " + err);
    }
  }

  function stopCamera() {
    if (mediaStream) {
      mediaStream.getTracks().forEach(track => track.stop());
      video.srcObject = null;
      mediaStream = null;
      isCameraOn = false;
      toggleBtn.textContent = " เปิดกล้อง";
    }
  }

  function toggleCamera() {
    if (isCameraOn) {
      stopCamera();
    } else {
      startCamera();
    }
  }

  function capture() {
    const phone = document.getElementById('phone').value;
    if (!phone) {
      alert("กรุณากรอกเบอร์โทรก่อน");
      return;
    }

    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;
    const ctx = canvas.getContext('2d');
    ctx.drawImage(video, 0, 0);
    canvas.toBlob(blob => {
      const formData = new FormData();
      formData.append('phone', phone);
      formData.append('image', blob, 'face.png');
      fetch('/', { method: 'POST', body: formData })
        .then(res => res.text())
        .then(html => { document.open(); document.write(html); document.close(); })
        .catch(err => alert("Upload failed: " + err));
    }, 'image/png');
  }

  // ไม่เปิดกล้องอัตโนมัติ
</script>
{% if request.args.get('msg') == 'deleted' %}
    <p style="color: green;">บัญชีของคุณถูกลบเรียบร้อยแล้ว</p>
{% endif %}
{% if request.args.get('msg') == 'registered' %}
  <p style="color: green;">ลงทะเบียนสำเร็จแล้ว</p>
{% endif %}
</body>
</html>
"""




ส่วนที่ 4 หน้า register

In [ ]:
#เรียกหน้า
@app.route('/register', methods=['GET'])
def register_get():
    return render_template_string(REGISTER_HTML)
#ส่งข้อมูล
@app.route('/register', methods=['POST'])
def register_post():
    name = request.form['name']
    surname = request.form['surname']
    phone = request.form['phone']

    # ตรวจสอบว่ามีเบอร์โทรศัพท์นี้อยู่แล้วหรือไม่
    df = pd.read_csv(csv_file)
    # Convert phone column to string to ensure correct comparison
    df['phone'] = df['phone'].astype(str)
    if phone in df['phone'].values:
        return " เบอร์โทรศัพท์นี้ถูกใช้แล้ว", 400

    user_folder = f"{name}_{surname}".replace(" ", "_")
    save_path = os.path.join(user_img_dir, user_folder)
    os.makedirs(save_path, exist_ok=True)

    # บันทึกภาพ 15 รูป
    for i in range(15):
        file = request.files.get(f'image_{i}')
        if file:
            file.save(os.path.join(save_path, f'img_{i+1}.jpg'))
        else:
            return f" ไม่พบภาพที่ {i+1}", 400

    # บันทึกข้อมูลลง CSV
    df.loc[len(df.index)] = [name, surname, phone, save_path]
    df.to_csv(csv_file, index=False)

    # โหลดข้อมูลทั้งหมดก่อน
    all_images, all_labels, label_map = load_data_rgb(user_img_dir)

    # ตรวจสอบว่ามีข้อมูลหรือไม่ก่อนที่จะแบ่ง
    if len(all_images) > 0:
        # ใช้ train_test_split
        trainX, testX, trainY, testY = train_test_split(
            all_images, all_labels, test_size=0.3, random_state=32, stratify=all_labels
        )
    else:
        # กรณีไม่มีข้อมูล ให้จัดการตามที่เหมาะสม
        return " ไม่มีข้อมูลภาพในระบบ"

    # สร้างโมเดล
    num_classes = len(label_map)
    input_layer = tf.keras.Input(shape=(100, 100, 3))
    model = Sequential([
        input_layer,
        Conv2D(32, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.3),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.3),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.4),
        Conv2D(256, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.4),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # เทรน
    model.fit(trainX, trainY, validation_split=0.2, epochs=10, batch_size=32)

    # ประเมินผล
    loss, acc = model.evaluate(testX, testY, verbose=0)
    print(f" Accuracy: {acc:.2f}")

    # บันทึกโมเดลและ label_map
    model_dir = f'{base_path}/model'
    os.makedirs(model_dir, exist_ok=True)
    model.save(f'{model_dir}/face_cnn_model.keras')
    with open(f'{model_dir}/label_map.json', 'w') as f:
        json.dump(label_map, f)
#พากลับหน้าหลัก
    return redirect(url_for('index', msg='ลงทะเบียนสำเร็จแล้ว'))


ส่วนที่ 5 หน้า login

In [ ]:
def preprocess_image(image_bytes):
    img = Image.open(image_bytes).convert("RGB").resize((100, 100))
    img = np.array(img) / 255.0
    return np.expand_dims(img, axis=0)

@app.route('/', methods=['GET', 'POST'])

def index():

    users_df = pd.read_csv(f'{user_img_dir}/users.csv')
    if request.method == 'POST':
      # ========== โหลดโมเดล CNN ==========
        model = tf.keras.models.load_model(f'{base_path}/model/face_cnn_model.keras')

        # ========== โหลด label_map ==========
        with open(f'{base_path}/model/label_map.json', 'r') as f:
            label_map = json.load(f)
        inv_label_map = {v: k for k, v in label_map.items()}
        phone = request.form['phone']
        file = request.files['image']

        user_row = users_df[users_df['phone'].astype(str) == phone]

        if user_row.empty:
            result = f"Login Failed: Phone {phone} not found."
            return render_template_string(HTML_TEMPLATE, result=result)

        else:
            expected_folder = os.path.basename(user_row.iloc[0]['folder'])
            img = preprocess_image(file)
            pred = model.predict(img)
            pred_label = inv_label_map[np.argmax(pred)]

            if pred_label == expected_folder:
                # Login สำเร็จเด้งเข้า Profile
                user_folder_name = os.path.basename(user_row.iloc[0]['folder'])
                return redirect(url_for('profile', user_folder=user_folder_name))
            else:
                result = f"Login Failed: Face not match"
                return render_template_string(HTML_TEMPLATE, result=result)

    # For GET requests, just render the login page
    response = make_response(render_template_string(HTML_TEMPLATE, result=''))
    response.headers['Content-Type'] = 'text/html; charset=utf-8'
    return response

ส่วนที่ 6 หน้า Profile

In [ ]:
# Add this route to serve static files (images)
@app.route('/static/uploads/UserImages/<user_folder>/<image_name>')
def uploaded_file(user_folder, image_name):
    return send_from_directory(os.path.join(user_img_dir, user_folder), image_name)

In [ ]:
@app.route('/profile/<user_folder>')
def profile(user_folder):
    # โหลดข้อมูลผู้ใช้
    users_df = pd.read_csv(csv_file)
    user_row = users_df[users_df['folder'].str.endswith(user_folder)]

    if user_row.empty:
        return "ไม่พบข้อมูลผู้ใช้", 404

    user_data = user_row.iloc[0]
    user_name = user_data['name']
    user_surname = user_data['surname']
    user_phone = user_data['phone']
    user_folder_name = os.path.basename(user_data['folder']) # Get just the folder name

    # ดึงรายการรูปภาพจากโฟลเดอร์ผู้ใช้
    user_images_path = os.path.join(user_img_dir, user_folder_name)
    images = [img for img in os.listdir(user_images_path) if img.endswith(('.jpg', '.jpeg', '.png'))]
    images.sort() # เรียงลำดับรูปภาพ

    # เรนเดอร์เทมเพลต HTML ของโปรไฟล์
    return render_template_string(PROFILE_HTML, user={'name': user_name, 'surname': user_surname, 'phone': user_phone, 'folder': user_folder_name}, images=images)

ส่วนที่ 7 การลบ USER

In [ ]:
@app.route('/delete_account', methods=['POST'])
def delete_account():
    phone_to_delete = request.form['phone']

    df = pd.read_csv(csv_file)
    user_row = df[df['phone'].astype(str) == phone_to_delete]

    if user_row.empty:
        return " ไม่พบผู้ใช้"

    user_folder = user_row.iloc[0]['folder']
    user_folder_name = os.path.basename(user_folder)
    full_path = os.path.join(user_img_dir, user_folder_name)

    # ลบโฟลเดอร์
    if os.path.exists(full_path):
        shutil.rmtree(full_path)

    # ลบจาก CSV
    df = df[df['phone'].astype(str) != phone_to_delete]
    df.to_csv(csv_file, index=False)

    # รีเทรนใหม่
    all_images, all_labels, label_map = load_data_rgb(user_img_dir)
    if len(all_images) == 0:
        return " ลบบัญชีเรียบร้อย ระบบไม่มีผู้ใช้คงเหลือ"

    trainX, testX, trainY, testY = train_test_split(
        all_images, all_labels, test_size=0.3, random_state=42, stratify=all_labels
    )
    # สร้างโมเดล
    num_classes = len(label_map)
    input_layer = tf.keras.Input(shape=(100, 100, 3))
    model = Sequential([
        input_layer,
        Conv2D(32, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.3),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.3),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.4),
        Conv2D(256, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.4),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(trainX, trainY, validation_split=0.2, epochs=10, batch_size=32)
    model.save(f'{base_path}/model/face_cnn_model.keras')
    with open(f'{base_path}/model/label_map.json', 'w') as f:
        json.dump(label_map, f)
#พากลับหน้าหลัก
    return redirect(url_for('index', msg='deleted'))


ส่วนที่ 8 รันหน้าเว็บ

In [ ]:
#  ตั้งค่า ngrok auth token
from google.colab import userdata
NGROK_AUTH_TOKEN = userdata.get("AUTH_TOKEN")
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
public_url = ngrok.connect(5000)
print(f" Public URL: {public_url}")

app.run(port=5000)

SecretNotFoundError: Secret 2t6qusTYKffWmgxQE0TLHEfK4Oh_4DtvXLRQuwSJqbsGd6Yib does not exist.